In [8]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
from imports import *
from functions import *

In [10]:
path_postgresql_creds = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\postgresql_creds.json"

with open(path_postgresql_creds, 'r') as file:
    content = json.load(file)
    user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]

db = "MyProjects"
schema = "End_To_End_Oceanography_ML"

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")
conn = engine.connect()

In [11]:
import json
import pandas as pd
from sqlalchemy import MetaData

# Charger les métadonnées du schéma existant
metadata = MetaData(schema=schema)
print("🔍 Chargement des métadonnées du schéma...")
metadata.reflect(bind=conn)
print("✅ Métadonnées chargées avec succès.")

# Récupérer les noms des tables
table_names = [t.name for t in metadata.sorted_tables]
print(f"🔢 Nombre de tables dans le schéma : {len(table_names)}")

# Filtrer les tables en fonction de leur préfixe
marine_tables = {t for t in table_names if t.startswith("bronze_marine_data_station")}
meteo_tables = {t for t in table_names if t.startswith("bronze_meteo_data_station")}
buoys_data_table = {t for t in table_names if t == "buoys_datas"}

print(f"🌊 Nombre de tables marines trouvées : {len(marine_tables)}")
print(f"🌧️ Nombre de tables météo trouvées : {len(meteo_tables)}")
print(f"🐋 Nombre de tables de bouées trouvées : {len(buoys_data_table)}")

# Initialiser le dictionnaire qui va contenir les résultats
table_dict = {}

# Compteurs pour suivre le nombre de tables chargées avec succès
marine_data_count = 0
meteo_data_count = 0
buoys_data_count = 0

# Vérifier et récupérer les données de la table "buoys_datas"
if buoys_data_table:
    print("🔄 Chargement des données de la table 'buoys_datas'...")
    buoy_data_table_name = next(iter(buoys_data_table))
    
    # Récupérer les données pour cette table
    buoys_datas = fetch_table_data(schema=schema, conn=conn, table_name=buoy_data_table_name, as_df=True)

    if buoys_datas is not None:
        print("📦 Données récupérées pour 'buoys_datas'.")
        if isinstance(buoys_datas, str):  # Si JSON sous forme de string
            buoys_datas = pd.DataFrame(json.loads(buoys_datas))
        elif isinstance(buoys_datas, dict):  # Si dictionnaire
            buoys_datas = pd.DataFrame(buoys_datas)

        # Ajouter les données dans le dictionnaire sous la clé "buoys_datas"
        table_dict["buoys_datas"] = buoys_datas
        buoys_data_count += 1  # Augmenter le compteur de données bouées
        print(f"🌊🐋 Table 'buoys_datas' chargée avec succès! Total de bouées : {buoys_data_count}")
    else:
        print("⚠️ Aucun résultat trouvé dans 'buoys_datas'.")

# Associer les tables marine et meteo en fonction du station_id et récupérer leurs données
for marine in marine_tables:
    print(f"🔄 Chargement des données pour la table marine : {marine}...")
    # Extraire le station_id de la table marine
    station_id = marine.split("_")[4]
    
    # Initialiser la structure pour chaque station si elle n'existe pas encore
    if station_id not in table_dict:
        table_dict[station_id] = {}

    # Récupérer les données pour la table marine
    marine_data = fetch_table_data(schema=schema, conn=conn, table_name=marine, as_df=True)
    
    if marine_data is not None:
        print(f"📦 Données récupérées pour la station {station_id} (Marine).")
        if isinstance(marine_data, str):  # Si JSON sous forme de string
            marine_data = pd.DataFrame(json.loads(marine_data))
        elif isinstance(marine_data, dict):  # Si dictionnaire
            marine_data = pd.DataFrame(marine_data)

        # Ajouter les données marines au dictionnaire pour chaque station
        table_dict[station_id]["Marine DataFrame"] = marine_data
        marine_data_count += 1  # Augmenter le compteur de données marines
        print(f"🌊🐋 Données marines chargées pour la station {station_id}! Total des données marines : {marine_data_count}")
    else:
        print(f"⚠️ Aucun résultat trouvé pour la station {station_id} (Marine).")

for meteo in meteo_tables:
    print(f"🔄 Chargement des données pour la table météo : {meteo}...")
    # Extraire le station_id de la table meteo
    station_id = meteo.split("_")[4]
    
    # Initialiser la structure pour chaque station si elle n'existe pas encore
    if station_id not in table_dict:
        table_dict[station_id] = {}

    # Récupérer les données pour la table meteo
    meteo_data = fetch_table_data(schema=schema, conn=conn, table_name=meteo, as_df=True)
    
    if meteo_data is not None:
        print(f"📦 Données récupérées pour la station {station_id} (Météo).")
        if isinstance(meteo_data, str):  # Si JSON sous forme de string
            meteo_data = pd.DataFrame(json.loads(meteo_data))
        elif isinstance(meteo_data, dict):  # Si dictionnaire
            meteo_data = pd.DataFrame(meteo_data)

        # Ajouter les données météorologiques au dictionnaire pour chaque station
        table_dict[station_id]["Meteo DataFrame"] = meteo_data
        meteo_data_count += 1  # Augmenter le compteur de données météo
        print(f"🌧️🌤️ Données météorologiques chargées pour la station {station_id}! Total des données météo : {meteo_data_count}")
    else:
        print(f"⚠️ Aucun résultat trouvé pour la station {station_id} (Météo).")

# Finalement, afficher un récapitulatif global
print(f"⭐🏆 Chargement des données terminé!")
print(f"🔢 Total des données bouées chargées : {buoys_data_count}")
print(f"🌊🐋 Total des données marines chargées : {marine_data_count}")
print(f"🌧️🌤️ Total des données météorologiques chargées : {meteo_data_count}")


🔍 Chargement des métadonnées du schéma...
✅ Métadonnées chargées avec succès.
🔢 Nombre de tables dans le schéma : 33
🌊 Nombre de tables marines trouvées : 16
🌧️ Nombre de tables météo trouvées : 16
🐋 Nombre de tables de bouées trouvées : 1
🔄 Chargement des données de la table 'buoys_datas'...
📦 Données récupérées pour 'buoys_datas'.
🌊🐋 Table 'buoys_datas' chargée avec succès! Total de bouées : 1
🔄 Chargement des données pour la table marine : bronze_marine_data_station_51002_southwest_hawaii_17-07_-157-75...
📦 Données récupérées pour la station 51002 (Marine).
🌊🐋 Données marines chargées pour la station 51002! Total des données marines : 1
🔄 Chargement des données pour la table marine : bronze_marine_data_station_46053_east_santa_barbara_34-241_-119...
📦 Données récupérées pour la station 46053 (Marine).
🌊🐋 Données marines chargées pour la station 46053! Total des données marines : 2
🔄 Chargement des données pour la table marine : bronze_marine_data_station_51000_northern_hawaii_one_23

In [12]:
table_dict["buoys_datas"].head()

,id,Station ID,Station Name,Zone,Lat,Lon,Bronze Marine Table Name,Bronze Meteo Table Name,Marine Dataframe,Meteo Dataframe
0,1,44020,station 44020 (llnr 13700),nantucket sound,41.497,-70.283,bronze_marine_data_station_44020_nantucket_sou...,bronze_meteo_data_station_44020_nantucket_soun...,"{""wind_direction"":{""0"":20.0,""1"":40.0,""2"":30.0,...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
1,2,44025,station 44025 (llnr 830),long island,40.258,-73.175,bronze_marine_data_station_44025_long_island_4...,bronze_meteo_data_station_44025_long_island_40...,"{""wind_direction"":{""0"":80.0,""1"":70.0,""2"":60.0,...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
2,3,44065,station 44065 (llnr 725),new york harbor entrance,40.368,-73.701,bronze_marine_data_station_44065_new_york_harb...,bronze_meteo_data_station_44065_new_york_harbo...,"{""wind_direction"":{""0"":120.0,""1"":120.0,""2"":120...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
3,4,46006,station 46006 (llnr 510),southeast papa,40.764,-137.377,bronze_marine_data_station_46006_southeast_pap...,bronze_meteo_data_station_46006_southeast_papa...,"{""wind_direction"":{""0"":270.0,""1"":280.0,""2"":270...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
4,5,46014,station 46014 (llnr 445),pt arena,39.225,-123.98,bronze_marine_data_station_46014_pt_arena_39-2...,bronze_meteo_data_station_46014_pt_arena_39-22...,"{""wind_direction"":{""0"":140.0,""1"":150.0,""2"":140...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."


Get Dataframes in Marine and Meteo Lists

In [13]:
df_meteo = table_dict["46006"]["Meteo DataFrame"]         
df_meteo.head()

,id,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,1,2024-12-17 01:00:00+01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0
1,2,2024-12-17 02:00:00+01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0
2,3,2024-12-17 03:00:00+01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
3,4,2024-12-17 04:00:00+01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
4,5,2024-12-17 05:00:00+01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0


In [14]:
df_marine = table_dict["46006"]["Marine DataFrame"]

In [27]:
list_silver_merged_df = []  # Liste pour stocker les DataFrames fusionnés
list_failed_dfs = []        # Liste pour stocker les couples de DataFrames échoués
dic_df = {}

conversions = 0

# Boucle sur chaque station du dictionnaire
for idx, (station_id, tables) in enumerate(table_dict.items()):
    print(f"\n🔄 Traitement de la station {station_id} ({idx + 1}/{len(table_dict)})...")
    
    dic_df[station_id] = {}
    
    # Vérification de la station "buoys_datas" qui doit être exclue du traitement normal
    if station_id != "buoys_datas":
        # Récupération des DataFrames météo et marins pour la station
        df_meteo = table_dict[station_id].get("Meteo DataFrame", None)
        df_marine = table_dict[station_id].get("Marine DataFrame", None)
    df_buoys_data = table_dict.get("buoys_datas", None)
    
    # Vérifier si toutes les données nécessaires sont présentes
    if df_meteo is None or df_marine is None or df_buoys_data is None:
        print(f"⚠️ Données manquantes pour la station {station_id}.")
        list_failed_dfs.append(station_id)
        continue
    
    print(f"📦 Données récupérées : Météo, Marine et Bouées pour la station {station_id}.")
    
    # Fusionner les coordonnées de bouée (Lat/Lon) avec les données marines
    print("📍 Fusion des coordonnées de bouées (Lat/Lon) avec les données marines...")
    for idx, row in df_buoys_data.iterrows():
        if str(row['Station ID']) == str(station_id):
            df_marine["Lat"] = df_buoys_data["Lat"]
            df_marine["Lon"] = df_buoys_data["Lon"]
    
    print(f"🌐 Coordonnées (Lat/Lon) fusionnées pour la station {station_id}.")
    
    # Conversion des types de données dans les DataFrames
    print(f"🔄 Conversion des types de données pour la station {station_id} (Marine et Météo)...")
    df_marine = auto_convert(df_marine)
    df_meteo = auto_convert(df_meteo)
    print(f"✅ Conversion terminée pour la station {station_id}.")
    conversions +=1
    
    # Traitement et rééchantillonnage des données marines
    print(f"🔁 Traitement et rééchantillonnage des données pour la station {station_id} (Marine)...")
    df_marine = process_and_resample(df_marine, column_name='time')
    print(f"✅ Données marines rééchantillonnées pour la station {station_id}.")
    
    # Traitement et rééchantillonnage des données météo
    print(f"🔁 Traitement et rééchantillonnage des données pour la station {station_id} (Météo)...")
    df_meteo = process_and_resample(df_meteo, column_name='date')
    print(f"✅ Données météorologiques rééchantillonnées pour la station {station_id}.")
    
    # Ajouter les DataFrames traitées dans le dictionnaire
    dic_df[station_id]["Marine Dataframe"] = df_marine
    dic_df[station_id]["Meteo Dataframe"] = df_meteo
    print(f"✔️ Données traitées et ajoutées au dictionnaire pour la station {station_id}.")
    
    # Fusionner les DataFrames Marine et Météo
    print(f"🔗 Fusion des données marines et météo pour la station {station_id}...")
    df_merged = pd.merge(df_marine, df_meteo, on='Datetime', how='outer')
    dic_df[station_id]["Merged Dataframe"] = df_merged

    print(f"💾 Données fusionnées ajoutées à la liste pour la station {station_id}.")
    
    # Ajouter le DataFrame fusionné à la liste des résultats
    list_silver_merged_df.append(dic_df[station_id])

 # Vérification de la dernière station pour stacker les DataFrames
print("🔀 Fusion finale de tous les DataFrames pour créer une grosse DataFrame...")
# Affichage de la taille des DataFrames avant la fusion finale
print(f"🔢 Nombre de DataFrames à fusionner : {len(list_silver_merged_df)}")

# Extraire uniquement les DataFrames fusionnés de chaque station
dataframes_to_concat = [station_data["Merged Dataframe"] for station_data in dic_df.values()]

# Fusionner les DataFrames
df_final = pd.concat(dataframes_to_concat, ignore_index=True)

# Imprimer quelques informations sur la fusion finale
print(f"📝 Taille de la DataFrame fusionnée : {df_final.shape}")
print("✅ Fusion finale effectuée avec succès !")


# Récapitulatif final
print("\n⭐🏆 Traitement terminé !")
print(f"🔢 Nombre total de stations traitées : {len(dic_df)}")
print(f"🔄 Conversions réussies : {conversions}")
print(f"❌ Nombre de stations échouées : {len(list_failed_dfs)}")
if list_failed_dfs:
    print(f"⚠️ Stations échouées : {', '.join(list_failed_dfs)}")



🔄 Traitement de la station buoys_datas (1/17)...
📦 Données récupérées : Météo, Marine et Bouées pour la station buoys_datas.
📍 Fusion des coordonnées de bouées (Lat/Lon) avec les données marines...
🌐 Coordonnées (Lat/Lon) fusionnées pour la station buoys_datas.
🔄 Conversion des types de données pour la station buoys_datas (Marine et Météo)...
✅ Conversion terminée pour la station buoys_datas.
🔁 Traitement et rééchantillonnage des données pour la station buoys_datas (Marine)...
Erreur : La colonne 'time' n'existe pas dans le DataFrame.
✅ Données marines rééchantillonnées pour la station buoys_datas.
🔁 Traitement et rééchantillonnage des données pour la station buoys_datas (Météo)...
Erreur : La colonne 'date' n'existe pas dans le DataFrame.
✅ Données météorologiques rééchantillonnées pour la station buoys_datas.
✔️ Données traitées et ajoutées au dictionnaire pour la station buoys_datas.
🔗 Fusion des données marines et météo pour la station buoys_datas...
💾 Données fusionnées ajoutées 

In [28]:
df_final.head()

,id_x,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,...,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility_y,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
# Afficher le nombre de valeurs manquantes (NaN) par colonne
for col in df_final.columns:
    print(f"Col '{col}' : {df_final[col].isna().sum()}/ {len(df_final)} ")
   

Col 'id_x' : 21709/ 132935 
Col 'wind_direction' : 22927/ 132935 
Col 'wind_speed' : 22269/ 132935 
Col 'wind_gust' : 22269/ 132935 
Col 'wave_height' : 76310/ 132935 
Col 'dominant_wave_period' : 99867/ 132935 
Col 'average_wave_period' : 76330/ 132935 
Col 'dominant_wave_direction' : 78269/ 132935 
Col 'pressure' : 22352/ 132935 
Col 'air_temperature' : 23862/ 132935 
Col 'water_temperature' : 29414/ 132935 
Col 'dewpoint' : 25132/ 132935 
Col 'visibility_x' : 132935/ 132935 
Col '3hr_pressure_tendency' : 114365/ 132935 
Col 'water_level_above_mean' : 132935/ 132935 
Col 'Station ID' : 21709/ 132935 
Col 'Lat' : 132935/ 132935 
Col 'Lon' : 132935/ 132935 
Col 'id_y' : 92543/ 132935 
Col 'temperature_2m' : 95705/ 132935 
Col 'relative_humidity_2m' : 95705/ 132935 
Col 'dew_point_2m' : 95705/ 132935 
Col 'precipitation' : 95705/ 132935 
Col 'rain' : 95705/ 132935 
Col 'showers' : 95705/ 132935 
Col 'pressure_msl' : 95705/ 132935 
Col 'surface_pressure' : 95705/ 132935 
Col 'cloud_cover

In [16]:
# def explore_dict_keys(d, parent_key='', sep='_'):
#     """
#     Explore un dictionnaire récursivement pour obtenir toutes les clés, y compris les sous-clés,
#     mais ne retourne pas les valeurs finales.

#     :param d: Le dictionnaire à explorer
#     :param parent_key: La clé parent qui est utilisée pour concaténer les sous-clés
#     :param sep: Le séparateur utilisé pour concaténer les clés (par défaut '_')
#     :return: Une liste des clés (et sous-clés)
#     """
#     keys = []
#     for k, v in d.items():
#         new_key = f"{parent_key}{sep}{k}" if parent_key else k
#         if isinstance(v, dict):  # Si la valeur est un dictionnaire, on explore récursivement
#             keys.append(new_key)  # Ajouter la clé, mais ne pas inclure la valeur
#             keys.extend(explore_dict_keys(v, new_key, sep=sep))  # Continuer l'exploration
#         else:
#             keys.append(new_key)  # Ajouter la clé finale
#     return keys

In [17]:
# def find_key_path(d, target_key, path=[]):
#     """
#     Recherche récursive d'une clé dans un dictionnaire et retourne son chemin.
#     :param d: dictionnaire
#     :param target_key: clé recherchée
#     :param path: liste pour stocker le chemin jusqu'à la clé
#     :return: chemin sous forme de liste
#     """
#     if isinstance(d, dict):  # Si le dictionnaire est encore imbriqué
#         for key, value in d.items():
#             new_path = path + [key]
#             if key == target_key:
#                 return new_path
#             elif isinstance(value, dict):
#                 result = find_key_path(value, target_key, new_path)
#                 if result:  # Si la clé est trouvée, retourner le chemin
#                     return result
#     return None  # Retourne None si la clé n'a pas été trouvée



# # Recherche du chemin pour la clé 'marine_data'
# path = find_key_path(table_dict, "Marine Dataframe")
# print(path)


Auto_convert Test

In [18]:
# for idx, (buoy_id, tables) in enumerate(table_dict.items()):  # Utilisation de .items() pour obtenir (clé, valeur)
#     if isinstance(tables, dict):
#         if idx == 1:  # Vérifier si l'index est égal à 1

In [19]:
key_list =[]
for key, value in dic_df.items():
    print(key)
    key_list.append(key)
key_list

buoys_datas
51002
46053
51000
46088
46014
46078
46084
46025
44065
46027
46086
46072
44025
44020
51001
46006


['buoys_datas',
 '51002',
 '46053',
 '51000',
 '46088',
 '46014',
 '46078',
 '46084',
 '46025',
 '44065',
 '46027',
 '46086',
 '46072',
 '44025',
 '44020',
 '51001',
 '46006']

Counting Rows of all Dataframes in total

In [20]:
full_merged_df.head()

NameError: name 'full_merged_df' is not defined

In [ ]:
for idx, row in full_merged_df.iterrows():
    if idx == 1:
        for col in full_merged_df.columns :
            print(f'{col} : {row[col]}')

Datetime : 2025-02-02 02:00:00+01:00
id_x : 6515.5
wind_direction : 270.0
wind_speed : 9.67
wind_gust : 11.5
wave_height : 1.45
dominant_wave_period : 11.0
average_wave_period : 4.55
dominant_wave_direction : 270.5
pressure : 1016.88
air_temperature : 14.47
water_temperature : 10.945
dewpoint : 13.42
3hr_pressure_tendency : -1.5
id_y : 1130.0
temperature_2m : 14.55
relative_humidity_2m : 82.0
dew_point_2m : 11.51
precipitation : 0.0
rain : 0.0
showers : 0.0
pressure_msl : 1018.0
surface_pressure : 1018.0
cloud_cover : 0.0
cloud_cover_low : 0.0
cloud_cover_mid : 0.0
cloud_cover_high : 0.0
visibility_y : 15200.0
wind_speed_10m : 40.85
soil_temperature_0cm : 16.29
soil_moisture_0_to_1cm : 0.11
is_day : 1.0


In [ ]:
full_merged_df = drop_columns_if_exist(df=full_merged_df, columns_to_drop=["id_x", 
    "id_y", "", "", "", "", "", "", "", "",])

In [ ]:
print(full_merged_df.dtypes)


Datetime                   datetime64[ns, UTC+01:00]
id_x                                         float64
wind_direction                               float64
wind_speed                                   float64
wind_gust                                    float64
wave_height                                  float64
dominant_wave_period                         float64
average_wave_period                          float64
dominant_wave_direction                      float64
pressure                                     float64
air_temperature                              float64
water_temperature                            float64
dewpoint                                     float64
3hr_pressure_tendency                        float64
id_y                                         float64
temperature_2m                               float64
relative_humidity_2m                         float64
dew_point_2m                                 float64
precipitation                                f